<a href="https://colab.research.google.com/github/kritikhurana2004/caseflow-management-system-SIH-/blob/main/CriminalCasesModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load your dataset (replace 'your_dataset.csv' with your actual dataset)
!wget https://www.dropbox.com/scl/fi/6vzbk9vz04rkhg5io9hz2/Dataset.csv?rlkey=xro0d8h4v3lbqp94yaec6p7as&dl=0
dataset = pd.read_csv('Dataset.csv?rlkey=xro0d8h4v3lbqp94yaec6p7as')

--2023-09-16 16:22:26--  https://www.dropbox.com/scl/fi/6vzbk9vz04rkhg5io9hz2/Dataset.csv?rlkey=xro0d8h4v3lbqp94yaec6p7as
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/e/scl/fi/6vzbk9vz04rkhg5io9hz2/Dataset.csv?rlkey=xro0d8h4v3lbqp94yaec6p7as [following]
--2023-09-16 16:22:27--  https://www.dropbox.com/e/scl/fi/6vzbk9vz04rkhg5io9hz2/Dataset.csv?rlkey=xro0d8h4v3lbqp94yaec6p7as
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd68698c531dcede28819942f53.dl.dropboxusercontent.com/cd/0/inline/CD0TF1MPhITSRmH2UCIR4kTwiJNimPrzFJUTwgO2xv6D6HXpxUajWkjVa8UtGavl38dhFfM-retZt8wSUhjSrv36VGjnBCW4NnbWK2vlbFICA44ObZl4sx2pGLAKAD1i_BR290Z1B4CrYjY7P2N0n-zj/file# [following]
--2023-09-16 16:22:28--  https://ucd68698c531dcede2881

In [ ]:
dataset.head()


,Case Description,Label
0,"Assault case resulting from a bar altercation,...",High Priority
1,Drug possession charges against an individual ...,High Priority
2,Burglary case with suspects captured on securi...,High Priority
3,"Tax evasion case against a local business, fin...",Manual Assistance
4,Vandalism charges against a minor for public p...,High Priority


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Ensure 'punkt' resource is downloaded (you've already done this)
nltk.download('punkt')

# Load your dataset containing case descriptions and their corresponding priority labels
# Replace 'your_dataset.csv' with the actual file path

# Check for missing values and drop rows with missing data
dataset.dropna(subset=['Case Description', 'Label'], inplace=True)

# Text cleaning function
def clean_text(text):
    # Remove non-alphabetic characters and digits
    text = text.str.replace('[^a-zA-Z\s]', '').str.replace('\d+', '')
    return text

# Tokenization and lemmatization function
def tokenize_and_lemmatize(text):
    # Ensure input is a valid string
    if isinstance(text, str):
        # Tokenize the sentence into words
        words = word_tokenize(text)

        # Initialize lemmatizer
        lemmatizer = WordNetLemmatizer()

        # Lemmatize words and join them back into a sentence
        lemmatized_text = ' '.join([lemmatizer.lemmatize(word) for word in words])

        return lemmatized_text
    else:
        return ''  # Return an empty string for non-string inputs

# Apply text cleaning to case descriptions
dataset['Case Description'] = clean_text(dataset['Case Description'])

# Apply tokenization and lemmatization to case descriptions
dataset['Cleaned_Description'] = dataset['Case Description'].apply(tokenize_and_lemmatize)

# Alternatively, you can use spaCy for tokenization and lemmatization
# Make sure to install spaCy and download the English language model
# import spacy
# nlp = spacy.load('en_core_web_sm')
# dataset['Cleaned_Description'] = dataset['Case Description'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    dataset['Case Description'],
    dataset['Label'],
    test_size=0.3,
    random_state=42
)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-4-cfc42be401a1>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('[^a-zA-Z\s]', '').str.replace('\d+', '')


LookupError: ignored

In [ ]:
print(dataset.head())

In [ ]:
print(dataset['Label'])


In [ ]:
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset and preprocess it as needed
# ...

# Define custom class weights based on class distribution
class_weights = [0.2, 0.5, 0.5, 0.5]  # Adjust weights as needed

# Convert class labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(dataset['Label'])

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset['Cleaned_Description'], y_encoded, test_size=0.2, random_state=42)

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(np.unique(y_encoded)))

# Tokenize and preprocess the text data for BERT
max_seq_length = 128  # Adjust the maximum sequence length as needed

X_train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=max_seq_length, return_tensors='pt', return_token_type_ids=False)
X_test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_seq_length, return_tensors='pt', return_token_type_ids=False)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Fine-tuning parameters (adjust as needed)
num_epochs = 3
batch_size = 16

# Define custom weights for the weighted sampler
class_sample_counts = [len(np.where(y_train == i)[0]) for i in range(len(np.unique(y_train)))]
weights = 1.0 / np.array(class_sample_counts)
sample_weights = weights[y_train]

# Create a WeightedRandomSampler
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# Create data loaders with the weighted sampler
train_dataset = TensorDataset(X_train_encodings['input_ids'], X_train_encodings['attention_mask'], torch.tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)

# Fine-tune BERT model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_inputs = {
        'input_ids': X_test_encodings['input_ids'].to(device),
        'attention_mask': X_test_encodings['attention_mask'].to(device)
    }
    logits = model(**test_inputs).logits
    predictions = torch.argmax(logits, dim=1)

# Convert predictions and test labels back to their original class labels
y_pred = label_encoder.inverse_transform(predictions.cpu().numpy())
y_true = label_encoder.inverse_transform(y_test)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')



In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset and preprocess it as needed
# ...

# Define custom class weights based on class distribution
class_weights = [0.2, 0.5, 0.5, 0.5]  # Adjust weights as needed

# Convert class labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(dataset['Label'])



# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset['Cleaned_Description'], y_encoded, test_size=0.2, random_state=42)



# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(np.unique(y_encoded)))

# Tokenize and preprocess the text data for BERT
max_seq_length = 128  # Adjust the maximum sequence length as needed

X_train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=max_seq_length, return_tensors='pt', return_token_type_ids=False)
X_test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_seq_length, return_tensors='pt', return_token_type_ids=False)

# Define custom weights for the weighted sampler
class_sample_counts = [len(np.where(y_train == i)[0]) for i in range(len(np.unique(y_train)))]
weights = 1.0 / np.array(class_sample_counts)
sample_weights = weights[y_train]

# Create a WeightedRandomSampler
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# Define hyperparameters to search over
learning_rates = [1e-5, 5e-5, 1e-4]
num_epochs_values = [3, 4, 5]
batch_sizes = [16, 32, 64]

best_accuracy = 0
best_hyperparameters = None

for learning_rate in learning_rates:
    for num_epochs in num_epochs_values:
        for batch_size in batch_sizes:
            # Define the optimizer
            optimizer = AdamW(model.parameters(), lr=learning_rate)

            # Create data loaders here (as shown in the previous code)
            train_dataset = TensorDataset(X_train_encodings['input_ids'], X_train_encodings['attention_mask'], torch.tensor(y_train))
            train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)

            # Fine-tune BERT model
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model.to(device)
            model.train()

            for epoch in range(num_epochs):
                total_loss = 0
                for batch in train_loader:
                    batch = tuple(t.to(device) for t in batch)
                    input_ids, attention_mask, labels = batch
                    optimizer.zero_grad()
                    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()

                print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.4f}')

            # Evaluate the model
            model.eval()
            with torch.no_grad():
                test_inputs = {
                    'input_ids': X_test_encodings['input_ids'].to(device),
                    'attention_mask': X_test_encodings['attention_mask'].to(device)
                }
                logits = model(**test_inputs).logits
                predictions = torch.argmax(logits, dim=1)

            # Convert predictions and test labels back to their original class labels
            y_pred = label_encoder.inverse_transform(predictions.cpu().numpy())
            y_true = label_encoder.inverse_transform(y_test)

            # Calculate accuracy
            accuracy = accuracy_score(y_true, y_pred)

            # Check if the current hyperparameters achieved a better accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hyperparameters = (learning_rate, num_epochs, batch_size)

print("Best Hyperparameters: ", best_hyperparameters)
print("Best Accuracy: ", best_accuracy)


In [ ]:
# Save the model and tokenizer
model.save_pretrained("CriminalCaseModel")
tokenizer.save_pretrained("CriminalCaseModel")


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained("CriminalCaseModel")
tokenizer = BertTokenizer.from_pretrained("CriminalCaseModel")

# Set the model to evaluation mode
model.eval()


In [ ]:
# Example of tokenizing and preprocessing a sentence
input_text = "theft"
input_encoding = tokenizer(input_text, truncation=True, padding=True, max_length=max_seq_length, return_tensors='pt')


In [ ]:
# Send input to the model and get predictions
with torch.no_grad():
    logits = model(**input_encoding).logits
    predictions = torch.argmax(logits, dim=1)


In [ ]:
# Convert predictions back to original class labels
predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

# Print the predicted label
print("Predicted Label:", predicted_labels[0])
